In [1]:
!git clone https://github.com/idiap/coqui-ai-TTS && cd coqui-ai-TTS && pip install -e .

Cloning into 'coqui-ai-TTS'...
remote: Enumerating objects: 30723, done.
remote: Counting objects: 100% (3266/3266), done.
remote: Compressing objects: 100% (274/274), done.
remote: Total 30723 (delta 3062), reused 3044 (delta 2991), pack-reused 27457 (from 1)
Receiving objects: 100% (30723/30723), 136.78 MiB | 26.55 MiB/s, done.
Resolving deltas: 100% (22237/22237), done.
Obtaining file:///kaggle/working/coqui-ai-TTS
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 45.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101

In [2]:
%cd coqui-ai-TTS

/kaggle/working/coqui-ai-TTS


In [3]:
%pwd

'/kaggle/working/coqui-ai-TTS'

In [4]:
import os
from trainer import Trainer, TrainerArgs
from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager

# Logging parameters
RUN_NAME = "GPT_XTTS_v2.0_English_FT"
PROJECT_NAME = "XTTS_trainer"
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None

# Set here the path that the checkpoints will be saved. Default: ./run/training/
OUT_PATH = os.path.join(os.getcwd(), "run", "training")

# Training Parameters
OPTIMIZER_WD_ONLY_ON_WEIGHTS = False  # for multi-gpu training please make it False
START_WITH_EVAL = True  # if True it will star with evaluation
BATCH_SIZE = 4  # set here the batch size
GRAD_ACUMM_STEPS = 64  # set here the grad accumulation steps
# Note: we recommend that BATCH_SIZE * GRAD_ACUMM_STEPS need to be at least 252 for more efficient training. You can increase/decrease BATCH_SIZE but then set GRAD_ACUMM_STEPS accordingly.

# Define here the dataset that you want to use for the fine-tuning on.
config_dataset = BaseDatasetConfig(
    formatter="ljspeech",
    dataset_name="English_Technical_dataset",
    path="/kaggle/input/technical-terms-dataset/Technical_dataset",
    meta_file_train="/kaggle/input/technical-terms-dataset/Technical_dataset/metadata.csv",
    language="en",
)

# Add here the configs of the datasets
DATASETS_CONFIG_LIST = [config_dataset]

# Define the path where XTTS v2.0.1 files will be downloaded
CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)

# DVAE files
DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

# Set the path to the downloaded files
DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

# download DVAE files if needed
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True)

# Download XTTS v2.0 checkpoint if needed
TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"

# XTTS transfer learning parameters: You we need to provide the paths of XTTS model checkpoint that you want to do the fine tuning.
TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))  # vocab.json file
#XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))  # model.pth file
XTTS_CHECKPOINT = "/kaggle/input/technical_finetune_english/pytorch/version-2/1/best_model_14490.pth"

# download XTTS v2.0 files if needed
if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True
    )

# Training sentences generations

SPEAKER_REFERENCE = [

    "/kaggle/input/technical-terms-dataset/Technical_dataset/wavs/audio_clip_10.wav"  # speaker reference to be used in training test sentences

]

LANGUAGE = config_dataset.language

def main():
    # init args and config
    model_args = GPTArgs(
        max_conditioning_length=132300,  # 6 secs
        min_conditioning_length=66150,  # 3 secs
        debug_loading_failures=False,
        #max_wav_length=255995,  # ~11.6 seconds
        #max_text_length=200,
        mel_norm_file=MEL_NORM_FILE,
        dvae_checkpoint=DVAE_CHECKPOINT,
        xtts_checkpoint=XTTS_CHECKPOINT,  # checkpoint path of the model that you want to fine-tune
        tokenizer_file=TOKENIZER_FILE,
        gpt_num_audio_tokens=1026,
        gpt_start_audio_token=1024,
        gpt_stop_audio_token=1025,
        gpt_use_masking_gt_prompt_approach=True,
        gpt_use_perceiver_resampler=True,
    )

    # define audio config
    audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=24000)
    # training parameters config
    config = GPTTrainerConfig(
        output_path=OUT_PATH,
        model_args=model_args,
        run_name=RUN_NAME,
        project_name=PROJECT_NAME,
        run_description="""
            GPT XTTS training
            """,
        dashboard_logger=DASHBOARD_LOGGER,
        logger_uri=LOGGER_URI,
        audio=audio_config,
        batch_size=BATCH_SIZE,
        batch_group_size=48,
        eval_batch_size=BATCH_SIZE,
        num_loader_workers=8,
        eval_split_max_size=256,
        print_step=50,
        plot_step=100,
        log_model_step=1000,
        save_step=5000,
        epochs = 21,
        #save_n_checkpoints=1,
        save_checkpoints=False,
        # target_loss="loss",
        print_eval=False,
        # Optimizer values like tortoise, pytorch implementation with modifications to not apply WD to non-weight parameters.
        optimizer="AdamW",
        optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
        optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
        lr=5e-06,  # learning rate
        lr_scheduler="MultiStepLR",
        # it was adjusted accordly for the new step scheme
        lr_scheduler_params={"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1},
        test_sentences=[

            {

                "text": "CUDA programming has a lot of scope in future.",

                "speaker_wav": SPEAKER_REFERENCE,

                "language": LANGUAGE,

            },

            {

                "text": "There are a lot of issues with the API call.",

                "speaker_wav": SPEAKER_REFERENCE,

                "language": LANGUAGE,

            },

        ],

    )

    # init the model from config
    model = GPTTrainer.init_from_config(config)

    # load training samples
    train_samples, eval_samples = load_tts_samples(
        DATASETS_CONFIG_LIST,
        eval_split=True,
        eval_split_max_size=config.eval_split_max_size,
        eval_split_size=config.eval_split_size,
    )

    # init the trainer and 🚀
    trainer = Trainer(
        TrainerArgs(
            restore_path=None,  # xtts checkpoint is restored via xtts_checkpoint key so no need of restore it using Trainer restore_path parameter
            skip_train_epoch=False,
            start_with_eval=START_WITH_EVAL,
            grad_accum_steps=GRAD_ACUMM_STEPS,
        ),
        config,
        output_path=OUT_PATH,
        model=model,
        train_samples=train_samples,
        eval_samples=eval_samples,
    )
    trainer.fit()

if __name__ == "__main__":

    main()


 > Downloading DVAE files!


  0%|          | 0.00/1.07k [00:00<?, ?iB/s]
100%|██████████| 1.07k/1.07k [00:00<00:00, 2.29kiB/s]

  1%|          | 1.11M/211M [00:00<00:19, 10.7MiB/s]
  1%|          | 2.18M/211M [00:00<00:19, 10.6MiB/s]
  2%|▏         | 3.24M/211M [00:00<00:19, 10.5MiB/s]
  2%|▏         | 4.29M/211M [00:00<00:19, 10.5MiB/s]
  3%|▎         | 5.34M/211M [00:00<00:19, 10.5MiB/s]
  3%|▎         | 6.39M/211M [00:00<00:19, 10.5MiB/s]
  4%|▎         | 7.44M/211M [00:00<00:19, 10.5MiB/s]
  4%|▍         | 8.49M/211M [00:00<00:19, 10.5MiB/s]
  5%|▍         | 9.53M/211M [00:00<00:19, 10.4MiB/s]
  5%|▌         | 10.6M/211M [00:01<00:19, 10.4MiB/s]
  6%|▌         | 11.6M/211M [00:01<00:19, 10.4MiB/s]
  6%|▌         | 12.7M/211M [00:01<00:18, 10.5MiB/s]
  7%|▋         | 13.7M/211M [00:01<00:18, 10.4MiB/s]
  7%|▋         | 14.8M/211M [00:01<00:18, 10.4MiB/s]
  8%|▊         | 15.8M/211M [00:01<00:18, 10.5MiB/s]
  8%|▊         | 16.9M/211M [00:01<00:18, 10.4MiB/s]
  9%|▊         | 17.9M/211M [00:01<00:18, 10.4MiB/s]

 > Downloading XTTS v2.0 files!


100%|██████████| 211M/211M [00:20<00:00, 10.2MiB/s]
 90%|████████▉ | 324k/361k [00:00<00:00, 3.24MiB/s]
100%|██████████| 361k/361k [00:00<00:00, 699kiB/s] 

  0%|          | 1.76M/1.87G [00:00<01:46, 17.6MiB/s]
  0%|          | 6.43M/1.87G [00:00<00:53, 34.7MiB/s]
  1%|          | 11.1M/1.87G [00:00<00:46, 40.0MiB/s]
  1%|          | 15.2M/1.87G [00:00<00:45, 40.6MiB/s]
  1%|          | 19.6M/1.87G [00:00<00:44, 41.6MiB/s]
  1%|▏         | 23.8M/1.87G [00:00<00:44, 41.8MiB/s]
  2%|▏         | 28.1M/1.87G [00:00<00:43, 42.1MiB/s]
  2%|▏         | 32.3M/1.87G [00:00<00:43, 42.1MiB/s]
  2%|▏         | 36.6M/1.87G [00:00<00:43, 42.3MiB/s]
  2%|▏         | 40.8M/1.87G [00:01<00:43, 42.4MiB/s]
  2%|▏         | 45.2M/1.87G [00:01<00:42, 42.8MiB/s]
  3%|▎         | 49.6M/1.87G [00:01<00:42, 43.1MiB/s]
  3%|▎         | 53.9M/1.87G [00:01<00:42, 43.0MiB/s]
  3%|▎         | 58.2M/1.87G [00:01<00:42, 42.5MiB/s]
  3%|▎         | 62.4M/1.87G [00:01<00:42, 42.4MiB/s]
  4%|▎         | 66.7M/1.87G [00: